In [26]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI


llm = ChatOpenAI(temperature=0.1)
# 메모리에 보내온 메시지의 수를 저장
# 또한 limit에 도달한 순간 오래된 메시지들을 요약(summarize)한다.
# 그러므로 가장 최근의 대화(상호작용)내용을 계속 추적가능하다.
# 가장 최근 및 가장 오래 전에 주고 받은 메시지 모두 잊혀지지 않고 요약된다.
memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=100,  # 요약되기 전에 가능한 메시지 토큰 수의 최대값
    return_messages=True,
)


def add_message(input, output):
    memory.save_context({"input": input}, {"output": output})


def get_history():
    return memory.load_memory_variables({})


add_message(
    "안녕! 나는 앤더슨이야, 나는 서울에 살아",
    "와우 멋진데!",
)

In [27]:
add_message(
    "서울은 매우 아름다워",
    "나도 가고싶어!!",
)

In [28]:
add_message(
    "북한산은 서울에서 가장 높은 산이야",
    "대단히 멋있다.",
)

In [30]:
get_history()

{'history': [SystemMessage(content='The human greets the AI in Korean and introduces themselves as Anderson living in Seoul.'),
  AIMessage(content='와우 멋진데!'),
  HumanMessage(content='서울은 매우 아름다워'),
  AIMessage(content='나도 가고싶어!!'),
  HumanMessage(content='북한산은 서울에서 가장 높은 산이야'),
  AIMessage(content='대단히 멋있다.')]}